In [1]:
import requests
import pandas as pd
import time

# URL to the FeatureServer
url = "https://services.arcgis.com/NzlPQPKn5QF9v2US/ArcGIS/rest/services/IrishPlanningApplications/FeatureServer/0/query"

# Initialize parameters for pagination
offset = 0
record_count = 1000  # Number of records to fetch per request
all_attributes = []

print("About to begin fetching requests (this could take a few minutes)")

while True:
    # Update query parameters for pagination
    params = {
        # "where": "PlanningAuthority='Galway County Council'",
        "where": "1=1",
        "outFields": "*",
        "f": "json",
        "resultOffset": offset,
        "resultRecordCount": record_count
    }

    try:
        # Send a GET request to the server
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises a HTTPError for bad responses

        data = response.json()
        features = data.get('features', [])

        if not features:
            break  # Exit the loop if no more features are returned

        for feature in features:
            attributes = feature['attributes'] 

            # Check if there's geometry data and extract latitude and longitude
            geometry = feature.get('geometry', None)
            if geometry:
                # Assuming point data; adjust as necessary for other types like polygons
                latitude = geometry.get('y', None)
                longitude = geometry.get('x', None)
                
                # Add latitude and longitude to the attributes with new keys
                attributes['Latitude'] = latitude
                attributes['Longitude'] = longitude

            for key, value in list(attributes.items()):
                if "date" in key.lower() and value is not None:  # Check for "date" and ensure value is not None
                    try:
                        # Using milliseconds ('ms'). Adjust seconds ('s') if this doesn't work)
                        attributes[key] = pd.to_datetime(value, unit='ms', errors='coerce').strftime('%Y-%m-%d %H:%M:%S')
                    except ValueError:
                        # If there's a ValueError, it might not be a valid timestamp
                        print(f"Conversion failed for {key} with value {value}. Keeping original value.")
                    except TypeError:
                        # If there's a TypeError, the value might not be convertible (e.g., a string that's not a timestamp)
                        print(f"Type error for {key} with value {value}. Keeping original value.")
                    except OverflowError:
                        # Handle cases where the timestamp is out of bounds for pandas to_datetime()
                        print(f"Overflow error for {key} with value {value}. Keeping original value.")

            all_attributes.append(attributes)

        print("Received ", len(all_attributes), " records", end="\r", flush=True)
        
        # Update the offset for the next request
        offset += record_count

        # Sleep between requests to avoid hitting rate limits
        time.sleep(1)  # Adjust the sleep time as needed

    except requests.exceptions.HTTPError as e:
        # Check if the error is due to rate limiting
        if e.response.status_code == 429:
            print("Rate limit hit, waiting before retrying...")
            time.sleep(10)  # Longer sleep time upon hitting rate limit
        else:
            print(f"HTTP error occurred: {e}")
            break
    except requests.exceptions.RequestException as e:
        # For other types of request-related errors
        print(f"Error fetching data: {e}")
        break

# Create a DataFrame from the collected attributes
df = pd.DataFrame(all_attributes)

# Save the DataFrame to a CSV file
filename = "IrishPlanningApplications.csv"
df.to_csv(filename, index=False)
print(len(all_attributes), " records saved to ", filename)


About to begin fetching requests (this could take a few minutes)
Conversion failed for ExpiryDate with value 33556723200000. Keeping original value.
Conversion failed for ExpiryDate with value 33494515200000. Keeping original value.
Conversion failed for ExpiryDate with value 32474044800000. Keeping original value.
Conversion failed for ExpiryDate with value 32474044800000. Keeping original value.
Conversion failed for DecisionDueDate with value 253370764800000. Keeping original value.
Conversion failed for DecisionDueDate with value 253370764800000. Keeping original value.
Conversion failed for DecisionDueDate with value 253370764800000. Keeping original value.
Conversion failed for DecisionDueDate with value 253370764800000. Keeping original value.
Conversion failed for GrantDate with value 96134860800000. Keeping original value.
Conversion failed for ExpiryDate with value 96292540800000. Keeping original value.
Conversion failed for ReceivedDate with value 38365315200000. Keeping or

In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
df = pd.read_csv(filename)

# Display the first few rows of the DataFrame to verify it's loaded correctly
df.head()

NameError: name 'filename' is not defined

Simple test - analyze the distribution of planning application decisions (e.g., Granted, Refused) over the years

In [6]:
import plotly.graph_objects as go
import pandas as pd

# Assuming df is existing DataFrame

# Filter the DataFrame for rows where PlanningAuthority is 'Galway County Council'
filtered_df = df[df['PlanningAuthority'].isin(['Galway County Council', 'Mayo County Council'])].copy()
# Drop potential missing values
filtered_df.dropna(subset=['DecisionDate', 'PlanningAuthority'], inplace=True)
# Convert decision dates to datetime format in the filtered DataFrame
filtered_df['DecisionDate'] = pd.to_datetime(filtered_df['DecisionDate'])

# Extract the year from the DecisionDate
filtered_df['DecisionYear'] = filtered_df['DecisionDate'].dt.year

# Group by 'DecisionYear', 'PlanningAuthority', and 'Decision' and count
decision_counts = filtered_df.groupby(['DecisionYear', 'PlanningAuthority', 'Decision']).size().reset_index(name='Counts')

# Prepare a figure
fig = go.Figure()

# Define unique categories
years = decision_counts['DecisionYear'].unique()
authorities = decision_counts['PlanningAuthority'].unique()
decisions = decision_counts['Decision'].unique()

# Create bars
for authority in authorities:
    for decision in decisions:
        df_filtered = decision_counts[(decision_counts['PlanningAuthority'] == authority) & (decision_counts['Decision'] == decision)]
        
        fig.add_trace(go.Bar(
            x=[f"{year}-{authority}" for year in df_filtered['DecisionYear']],
            y=df_filtered['Counts'],
            name=f"{authority} - {decision}",
            text=df_filtered['Counts'],
            textposition='auto',
        ))

# Customize appearance
fig.update_layout(
    barmode='stack',
    title='Planning Application Decisions by Year for Galway and Mayo County Councils',
    xaxis_title='Year - Planning Authority',
    yaxis_title='Number of Decisions',
    legend_title='Decision by Authority',
)

# Show the plot
fig.show()


Now, simplify our analysis into a simple table focusing on Galway County Council only first. this aims to replicate the test analysis in this doc - https://docs.google.com/spreadsheets/d/1dAIAc0Sm136LR-1PBHJ3R2-9v4FHaC-xnABoVQQ4PPc/edit#gid=993444100

In [30]:
import pandas as pd

galway_df = df[df['PlanningAuthority'] == 'Galway County Council'].copy()
galway_df['Received Year'] = pd.to_datetime(galway_df['ReceivedDate']).dt.year

results_df = pd.DataFrame(columns=['Received Year', 'Conditional Rate', 'Refusal Rate', 'Unconditional Rate'])

rows_list = []

for year in sorted(galway_df['Received Year'].unique()):
    year_data = galway_df[galway_df['Received Year'] == year]
    total_decisions = len(year_data)
    
    conditional_rate = ((year_data['Decision'].str.strip() == 'CONDITIONAL').sum() / total_decisions * 100) if total_decisions > 0 else 0
    refusal_rate = ((year_data['Decision'].str.strip() == 'REFUSED').sum() / total_decisions * 100) if total_decisions > 0 else 0
    unconditional_rate = ((year_data['Decision'].str.strip() == 'UNCONDITIONAL').sum() / total_decisions * 100) if total_decisions > 0 else 0
    
    rows_list.append({
        'Received Year': year,
        'Conditional Rate': round(conditional_rate, 2),
        'Refusal Rate': round(refusal_rate, 2),
        'Unconditional Rate': round(unconditional_rate, 2)
    })

results_df = pd.DataFrame(rows_list)
results_df['Received Year'] = results_df['Received Year'].astype(int)


# Ensure 'Received Year' column is of type int
results_df['Received Year'] = results_df['Received Year'].astype(int)

# Display the DataFrame
print(results_df)

   Received Year  Conditional Rate  Refusal Rate  Unconditional Rate
0           2015             68.61          3.94                9.01
1           2016             70.82          5.03                5.08
2           2017             76.52          5.77                2.86
3           2018             79.07          9.57                2.87
4           2019             73.05         13.40                1.97
5           2020             73.51         13.48                1.98
6           2021             71.45         17.39                2.35
7           2022             66.95         22.89                0.79
8           2023             62.35         14.61                0.06
9           2024              8.13          1.06                0.00


Same analysis, for Mayo this time

In [31]:
mayo_df = df[df['PlanningAuthority'] == 'Mayo County Council'].copy()
mayo_df['Received Year'] = pd.to_datetime(mayo_df['ReceivedDate']).dt.year

results_df = pd.DataFrame(columns=['Received Year', 'Conditional Rate', 'Refusal Rate', 'Unconditional Rate'])

rows_list = []

for year in sorted(mayo_df['Received Year'].unique()):
    year_data = mayo_df[mayo_df['Received Year'] == year]
    total_decisions = len(year_data)
    
    conditional_rate = ((year_data['Decision'].str.strip() == 'CONDITIONAL').sum() / total_decisions * 100) if total_decisions > 0 else 0
    refusal_rate = ((year_data['Decision'].str.strip() == 'REFUSED').sum() / total_decisions * 100) if total_decisions > 0 else 0
    unconditional_rate = ((year_data['Decision'].str.strip() == 'UNCONDITIONAL').sum() / total_decisions * 100) if total_decisions > 0 else 0
    
    rows_list.append({
        'Received Year': year,
        'Conditional Rate': round(conditional_rate, 2),
        'Refusal Rate': round(refusal_rate, 2),
        'Unconditional Rate': round(unconditional_rate, 2)
    })

results_df = pd.DataFrame(rows_list)
results_df['Received Year'] = results_df['Received Year'].astype(int)


# Ensure 'Received Year' column is of type int
results_df['Received Year'] = results_df['Received Year'].astype(int)

# Display the DataFrame
print(results_df)

   Received Year  Conditional Rate  Refusal Rate  Unconditional Rate
0           2015             88.48          4.41                 0.0
1           2016             88.34          4.88                 0.0
2           2017             87.49          5.27                 0.0
3           2018             87.59          5.21                 0.0
4           2019             83.49          8.15                 0.0
5           2020             82.76          7.43                 0.0
6           2021             84.35          8.13                 0.0
7           2022             79.66          6.10                 0.0
8           2023             69.49          4.28                 0.0
9           2024              6.76          0.68                 0.0
